In [ ]:
import gpmultipy, scipy, string
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from gpmultipy import dataset

In [ ]:
import GPy

In [ ]:
%pylab

In [ ]:
%matplotlib inline

In [ ]:
def wellPosition(number):
    number = int(number)
    if number > 100:
        return wellPosition(number%100)
    if number == 0:
        return 9,9
    return [(number-1) % 10, (number-1)/10]

In [ ]:
ds = dataset.DataSet('data/')
# x,y,effect,labels = ds.build(Strain='ura3',mM_PQ=0.0,scale='range',Bio='F',effects=['Well'])
x,y,effect,labels = ds.build(Strain='ura3',mM_PQ=0.0,scale='range',effects=['Well'])
y = np.log2(y)

x = x[6:,:]
y = y[6:,:]

x = x[::10,:]
y = y[::10,:]

In [ ]:
position = np.array(map(wellPosition,labels[0]))
dist = np.column_stack((position.min(1),(9-position).min(1))).min(1)

In [ ]:
select = dist != 0
y = y[:,select]
y = (y-y.mean())/y.std()

In [ ]:
import scipy.interpolate

splines = [scipy.interpolate.splrep(x[:,0],y[:,i],s=0.1) for i in range(y.shape[1])]
dsplines = [scipy.interpolate.splder(spl) for spl in splines]

In [ ]:
[plt.plot(scipy.interpolate.splev(x[:,0], spl,der=0)) for spl in splines];

mu = np.mean([scipy.interpolate.splev(x[:,0], spl,der=0) for spl in splines], 0)
plt.plot(mu, c='k', lw=3)

In [ ]:
[plt.plot(scipy.interpolate.splev(x[:,0], dspl,der=0)) for dspl in dsplines];
dmu = np.mean([scipy.interpolate.splev(x[:,0], spl,der=1) for spl in splines], 0)

In [ ]:
[plt.plot(scipy.interpolate.splev(x[:,0], spl,der=0) - mu) for spl in splines];

In [ ]:
[plt.plot(scipy.interpolate.splev(x[:,0], dspl,der=0) - dmu) for dspl in dsplines];

In [ ]:
plt.figure(figsize=(10,6))

plt.subplot(221)
[plt.plot(scipy.interpolate.splev(x[:,0], spl,der=0)) for spl in splines];
mu = np.mean([scipy.interpolate.splev(x[:,0], spl,der=0) for spl in splines], 0)
plt.plot(mu, c='k', lw=3)
plt.title('growth splines: $f_i(x)$')

plt.subplot(222)
[plt.plot(scipy.interpolate.splev(x[:,0], dspl,der=0)) for dspl in dsplines];
dmu = np.mean([scipy.interpolate.splev(x[:,0], spl,der=1) for spl in splines], 0)
plt.plot(dmu, c='k', lw=3)
plt.title('growth-rate splines: $\partial f_i(x)/\partial x$')

plt.subplot(223)
[plt.plot(scipy.interpolate.splev(x[:,0], spl,der=0) - mu, alpha=.4) for spl in splines];
plt.title('$f_i(x) - \mu(x)$')

plt.subplot(224)
[plt.plot(scipy.interpolate.splev(x[:,0], dspl,der=0) - dmu, alpha=.4) for dspl in dsplines];
plt.title('$\partial f_i(x) / \partial x - \partial\mu(x)/\partial x$')

plt.tight_layout()
plt.savefig('figures/spline-hsal.pdf')

In [ ]:
plt.figure(figsize=(5*4, 4.*len(splines)/5))

mu = np.mean([scipy.interpolate.splev(x[:,0], spl,der=0) for spl in splines], 0)
dmu = np.mean([scipy.interpolate.splev(x[:,0], spl,der=1) for spl in splines], 0)

for i, spl in enumerate(splines):
    
    plt.subplot(len(splines)/5+1, 5, i+1)
    plt.plot(scipy.interpolate.splev(x[:,0], spl,der=0))
    plt.plot(mu, c='k', lw=3)
    plt.twinx()
    plt.plot(scipy.interpolate.splev(x[:,0], spl,der=1))
    plt.plot(dmu, c='k', lw=3)

In [ ]:
plt.figure(figsize=(5*4, 4.*len(splines)/5))

mu = np.mean([scipy.interpolate.splev(x[:,0], spl,der=0) for spl in splines], 0)
dmu = np.mean([scipy.interpolate.splev(x[:,0], spl,der=1) for spl in splines], 0)

for i, spl in enumerate(splines):
    
    plt.subplot(len(splines)/5+1, 5, i+1)
    plt.plot(scipy.interpolate.splev(x[:,0], spl,der=0) - mu)
    plt.plot([0, 12], [0,0], c='k', lw=2)
    plt.twinx()
    plt.plot(scipy.interpolate.splev(x[:,0], spl,der=1) - dmu, c='r')
    plt.plot([0, 12], [0,0], c='gray', lw=2)
    
plt.tight_layout()

In [ ]:
# y = y-y.mean(0)

In [ ]:
plt.plot(x,y,c='k',alpha=.4); plt.plot(x,y.mean(1),c='r',lw=3);
plt.xlabel("time (normalized)",fontsize=25)
plt.ylabel("OD (normalized)",fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

In [ ]:
plt.figure(figsize=(6,4))

plt.plot(x,y-y.mean(1)[:,None],c='k',alpha=.4)

plt.xlabel("time (normalized)",fontsize=25)
plt.ylabel("OD (normalized)",fontsize=25)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

In [ ]:
xgp = np.zeros((y.shape[0]*y.shape[1], 2))
xgp[:,0] = np.tile(x[:,0], y.shape[1])
xgp[:,1] = np.repeat(range(y.shape[1]), y.shape[0])

ygp = y.T.reshape((y.shape[0]*y.shape[1],1))

plt.scatter(xgp[:,0], ygp)

In [ ]:
xgp.shape

In [ ]:
m0 = GPy.models.GPRegression(xgp, ygp, GPy.kern.RBF(1))
m0.randomize()
m0.optimize()
m0

In [ ]:
m0.plot(fixed_inputs=[(1,0)])

In [ ]:
mu, cov = m0.predict(xgp, full_cov=True)

s = scipy.stats.multivariate_normal.rvs(mu[:,0],cov).reshape(y.shape[1],y.shape[0]).T

plt.figure(figsize=(6,6))
plt.subplot(221)
plt.plot(y);

plt.subplot(222)
plt.plot(s);

plt.subplot(223)
plt.plot(x,y-y.mean(1)[:,None],c='k',alpha=.4)

plt.subplot(224)
plt.plot(x,s-s.mean(1)[:,None],c='k',alpha=.4);

In [ ]:
mu, var = m0.predict_noiseless(xgp[:y.shape[0],:], full_cov=False)
mu = mu[:,0]
std = np.sqrt(var[:,0])

plt.plot(xgp[:y.shape[0],0],mu)
plt.fill_between(xgp[:y.shape[0],0],mu-1.96*std,mu+1.96*std,alpha=.5)

In [ ]:
# kern = GPy.kern.Hierarchical([GPy.kern.Matern32(1), GPy.kern.RBF(1)])
kern = GPy.kern.Hierarchical([GPy.kern.RBF(1,name='base'), GPy.kern.Matern32(1,name='replicate')])
# kern = GPy.kern.Hierarchical([GPy.kern.RBF(1), GPy.kern.RBF(1)])

m1 = GPy.models.GPRegression(xgp, ygp, kern)
m1.randomize()
m1.optimize()
m1

In [ ]:
xpred = xgp.copy()
xpred[:,1] += xpred[:,1].max() + 1

mu, cov = m1.predict(xpred, full_cov=True)

ylim = (y-y.mean(1)[:,None]).min(), (y-y.mean(1)[:,None]).max()

plt.figure(figsize=(8,8))
plt.subplot(2,2,1)
plt.plot(y);
plt.title("original data")


s = scipy.stats.multivariate_normal.rvs(mu[:,0],cov).reshape(y.shape[1],y.shape[0]).T
plt.subplot(2, 2, 2)
plt.plot(s);
plt.title("resample")

plt.subplot(2, 2, 4)
plt.plot(x,s-s.mean(1)[:,None],c='k',alpha=.4)
plt.ylim(ylim)

plt.subplot(2, 2,3)
plt.plot(x,y-y.mean(1)[:,None],c='k',alpha=.4);

plt.savefig("figures/m1-resample.pdf",bbox_inches='tight')

In [ ]:
xpred = xgp.copy()
xpred[:,1] += xpred[:,1].max() + 1

mu, cov = m1.predict(xpred, full_cov=True)

ylim = (y-y.mean(1)[:,None]).min(), (y-y.mean(1)[:,None]).max()

resamp = 5

plt.figure(figsize=(3+3*resamp,6))
plt.subplot(2,1+resamp,1)
plt.plot(y);
plt.title("original data")

for i in range(resamp):
    plt.title("resample %d"%(i+1))
    s = scipy.stats.multivariate_normal.rvs(mu[:,0],cov).reshape(y.shape[1],y.shape[0]).T
    plt.subplot(2, 1+resamp,2+i)
    plt.plot(s);
    
    plt.subplot(2, 1+resamp,2+resamp+1+i)
    plt.plot(x,s-s.mean(1)[:,None],c='k',alpha=.4)
    plt.ylim(ylim)

plt.subplot(2, 1+resamp, resamp+2)
plt.plot(x,y-y.mean(1)[:,None],c='k',alpha=.4);

plt.savefig("figures/m1-resample-%d.pdf"%resamp,bbox_inches='tight')

In [ ]:
mu, var = m0.predict_noiseless(xgp[:y.shape[0],:], full_cov=False)
mu = mu[:,0]
std = np.sqrt(var[:,0])


In [ ]:
def autocorr(x, t=1):
    return numpy.corrcoef(numpy.array([x[0:len(x)-t], x[t:len(x)]]))[0,1]

In [ ]:
xpred = xgp.copy()
xpred[:,1] += xpred[:,1].max() + 1

ylim = (y-y.mean(1)[:,None]).min(), (y-y.mean(1)[:,None]).max()

plt.figure(figsize=(9,4))
plt.subplot(3,3,1)
plt.plot(x,y,c='k',alpha=.2);
plt.title("original data")

plt.subplot(3, 3, 2)
mu, cov = m0.predict(xpred, full_cov=True)
s = scipy.stats.multivariate_normal.rvs(mu[:,0],cov).reshape(y.shape[1],y.shape[0]).T
plt.plot(x,s,c='k',alpha=.2);
plt.title("$M_0$")

plt.subplot(3, 3, 5)
plt.plot(x,s-s.mean(1)[:,None],c='k',alpha=.2)
plt.ylim(ylim)

plt.subplot(3,3,8)
cor = np.apply_along_axis(lambda x: [autocorr(x-s.mean(1), z) for z in range(1, s.shape[0]-2)], 0, s)
plt.boxplot(cor.T)
plt.xticks([])

plt.subplot(3, 3, 3)
mu, cov = m1.predict(xpred, full_cov=True)
s = scipy.stats.multivariate_normal.rvs(mu[:,0],cov).reshape(y.shape[1],y.shape[0]).T
plt.plot(x,s,c='k',alpha=.2);
plt.title("$M_1$")

plt.subplot(3, 3, 6)
plt.plot(x,s-s.mean(1)[:,None],c='k',alpha=.2)
plt.ylim(ylim)

plt.subplot(3,3,9)
cor = np.apply_along_axis(lambda x: [autocorr(x-s.mean(1), z) for z in range(1, s.shape[0]-2)], 0, s)
plt.boxplot(cor.T)
plt.xticks([])

plt.subplot(3, 3,4)
plt.plot(x,y-y.mean(1)[:,None],c='k',alpha=.2);

plt.subplot(3,3,7)
cor = np.apply_along_axis(lambda x: [autocorr(x-y.mean(1), z) for z in range(1, y.shape[0]-2)], 0, y)
plt.boxplot(cor.T)
plt.xticks([])

plt.tight_layout()
plt.savefig("figures/mAll.pdf",bbox_inches='tight')

In [ ]:
mu, cov = m1.predict(xpred, full_cov=True)
s = scipy.stats.multivariate_normal.rvs(mu[:,0],cov).reshape(y.shape[1],y.shape[0]).T
cor = np.apply_along_axis(lambda x: [autocorr(x-s.mean(1), z) for z in range(1, s.shape[0]-2)], 0, s)

In [ ]:
i,lag = 0, 1

plt.boxplot(cor.T)
plt.plot(range(1, x.shape[0]-1),[cov[i,i+lag]/np.product(np.sqrt(cov.diagonal()[[i,i+lag]])) for lag in range(2,x.shape[0]-i)])

In [ ]:
xpred = xgp.copy()
xpred[:,1] += xpred[:,1].max() + 1

ylim = (y-y.mean(1)[:,None]).min(), (y-y.mean(1)[:,None]).max()

plt.figure(figsize=(9,9))
plt.subplot(2,3,1)
plt.plot(x,y,c='k',alpha=.2);
plt.title("original data")

plt.subplot(2, 3, 2)
mu, cov = m0.predict(xpred, full_cov=True)
s = scipy.stats.multivariate_normal.rvs(mu[:,0],cov).reshape(y.shape[1],y.shape[0]).T
plt.plot(x,s,c='k',alpha=.2);
plt.title("m0")

plt.subplot(2, 3, 5)
plt.plot(x,s-s.mean(1)[:,None],c='k',alpha=.2)
plt.ylim(ylim)

plt.subplot(2, 3, 3)
mu, cov = m1.predict(xpred, full_cov=True)
s = scipy.stats.multivariate_normal.rvs(mu[:,0],cov).reshape(y.shape[1],y.shape[0]).T
plt.plot(x,s,c='k',alpha=.2);
plt.title("m1")

plt.subplot(2, 3, 6)
plt.plot(x,s-s.mean(1)[:,None],c='k',alpha=.2)
plt.ylim(ylim)

plt.subplot(2, 3,4)
plt.plot(x,y-y.mean(1)[:,None],c='k',alpha=.2);

plt.savefig("figures/mAll-resample.pdf",bbox_inches='tight')

In [ ]:
mu, cov = m1.predict(xgp, full_cov=True)

s = scipy.stats.multivariate_normal.rvs(mu[:,0],cov).reshape(y.shape[1],y.shape[0]).T

plt.subplot(121)
plt.plot(y);

plt.subplot(122)
plt.plot(s);

In [ ]:
m1.plot(fixed_inputs=[(1,-1)],which_data_rows=[])

m1.plot_f(fixed_inputs=[(1,-1)])

In [ ]:
xpred = np.zeros((50,2))
xpred[:,0] = np.linspace(xgp[:,0].min(), xgp[:,0].max())

plt.figure(figsize=(10,10))

mu, var = m0.predict_noiseless(xpred, full_cov=False)
mu = mu[:,0]
std = np.sqrt(var[:,0])

plt.plot(xpred[:,0],mu)
plt.fill_between(xpred[:,0],mu-1.96*std,mu+1.96*std,alpha=.5)

mu, var = m1.predict_noiseless(xpred, full_cov=False, kern=m1.kern.base)
mu = mu[:,0]
std = np.sqrt(var[:,0])

plt.plot(xpred[:,0],mu)
plt.fill_between(xpred[:,0],mu-1.96*std,mu+1.96*std,alpha=.5)

In [ ]:
plt.bar(range(2), [m0.log_likelihood(), m1.log_likelihood()])
plt.xticks(range(2),['M0', 'M1'])
plt.title("log-likelihood",fontsize=20)

plt.savefig('figures/hsal-loglikelihood.pdf', bbox_inches='tight')

In [ ]:
xpred = np.zeros((50,2))
xpred[:,0] = np.linspace(xgp[:,0].min(), xgp[:,0].max())

plt.figure(figsize=(10,10))

mu, var0 = m0.predict_noiseless(xpred, full_cov=False)
mu = mu[:,0]
std0 = np.sqrt(var0[:,0])

mu, var1 = m1.predict_noiseless(xpred, full_cov=False, kern=m1.kern.base)
mu = mu[:,0]
std1 = np.sqrt(var1[:,0])

plt.boxplot([1.98*std0, 1.98*std1]);
# plt.semilogy()
plt.semilogy(basey=2)
plt.yticks([2.**(-i) for i in range(4,7)])

In [ ]:
xpred = np.zeros((50,2))
xpred[:,0] = np.linspace(xgp[:,0].min(), xgp[:,0].max())

mu1, var1 = m0.predict_noiseless(xpred, full_cov=False)
mu1 = mu1[:,0]

mu2, var2 = m1.predict_noiseless(xpred, full_cov=False, kern=m1.kern.base)
mu2 = mu2[:,0]

# plt.plot(var1)
# plt.plot(var2)
plt.plot(var1/var2)

In [ ]:
m0.kern.variance, m1.kern.base.variance, m0.likelihood.variance, m1.likelihood.variance

In [ ]:
plt.scatter(range(4), [m0.kern.variance, m1.kern.base.variance, m0.likelihood.variance, m1.likelihood.variance])

plt.semilogy()

In [ ]:
m0.kern.variance / (m0.kern.variance + m0.likelihood.variance)

In [ ]:
m1.kern.base.variance / (m1.kern.base.variance + m1.kern.replicate.variance + m1.likelihood.variance)